# TP 3 

#### Importer pymongo

In [38]:
from pymongo import MongoClient

#### Connexion avec la base de données

In [39]:
client = MongoClient('mongodb://localhost:27017/')
db = client.db_agg
restaurants = db.restaurants
print('connexion success')

connexion success


### Q : La répartition des restaurants par quartier ?

In [40]:
repart_resto_borough = { "$group":{"_id": "$borough","nb_restaurant":{"$sum" : 1}}}
tri = {"$sort" : {"_id":1}}
for rest in restaurants.aggregate([repart_resto_borough, tri]):
    print(rest) 

{'_id': 'Bronx', 'nb_restaurant': 2338}
{'_id': 'Brooklyn', 'nb_restaurant': 6085}
{'_id': 'Manhattan', 'nb_restaurant': 10258}
{'_id': 'Missing', 'nb_restaurant': 51}
{'_id': 'Queens', 'nb_restaurant': 5656}
{'_id': 'Staten Island', 'nb_restaurant': 969}


### Q : La répartition des restaurants dont la dernière inspection a donné un grade "C" par quartier ?

In [41]:
match_grade_C = { "$match" : {"grades.0.grade":"C"}}
proj = {"$project" : {"name":1, "borough":1,"_id":0}}
tri = {"$sort" : {"_id":1}}
for grad in restaurants.aggregate([match_grade_C,proj,repart_resto_borough, tri]):
    print(grad)

{'_id': 'Bronx', 'nb_restaurant': 27}
{'_id': 'Brooklyn', 'nb_restaurant': 56}
{'_id': 'Manhattan', 'nb_restaurant': 83}
{'_id': 'Queens', 'nb_restaurant': 47}
{'_id': 'Staten Island', 'nb_restaurant': 7}


### Q : Calculer le score moyen des resto par quartier et trier par score décroissant ?

In [42]:

eclater_liste = {"$unwind" : "$grades"}
mean_quartier = {"$group" : {"_id":"$borough","moyenne":{"$avg":"$grades.score"}}}
tri = {"$sort": {"moyenne":-1}}
for grad in restaurants.aggregate([eclater_liste, mean_quartier, tri]):
    print(grad)

{'_id': 'Queens', 'moyenne': 11.634865110930088}
{'_id': 'Brooklyn', 'moyenne': 11.447723132969035}
{'_id': 'Manhattan', 'moyenne': 11.41823125728344}
{'_id': 'Staten Island', 'moyenne': 11.370957711442786}
{'_id': 'Bronx', 'moyenne': 11.036186099942562}
{'_id': 'Missing', 'moyenne': 9.632911392405063}


# Exercice  Aggréger des transactions   

### Q : Créer une collection "transactions" à partir du fichier transactions.json et répondre aux questions suivantes

### Q-1. Calculer le montant total des paiements ?

In [68]:
transactions = db.transactions
succes = { "$match" : {"Payment.Success":True}}
somme = {"$group" : {"_id":"total", "Total":{"$sum":"$Payment.Total"}}}
for payment in transactions.aggregate([succes,somme]):
    print(payment)

{'_id': 'total', 'Total': 1230}


### Q-2. Calculer le montant total par transactions (montant total du panier) ?

In [67]:
unwind = {"$unwind" : "$Panier"}

somme = {"$group" : {"_id":"$TransactionId", "Total":{"$sum":"$Panier.price"}}}
for paniers in transactions.aggregate([unwind,somme]):
    print(paniers)

{'_id': 'tran4', 'Total': 410}
{'_id': 'tran3', 'Total': 410}
{'_id': 'tran2', 'Total': 410}
{'_id': 'tran1', 'Total': 410}


### Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?

In [66]:
transactions = db.transactions
succes = { "$match" : {"Payment.Success":True}}
somme = {"$group" : {"_id":"$Payment.Type", "Total":{"$sum":"$Payment.Total"}}}
for payment in transactions.aggregate([succes,somme]):
    print(payment)

{'_id': 'Credit-Card', 'Total': 820}
{'_id': 'Debit-Card', 'Total': 410}


### Q-4. Trouver l'Id le plus élevé.

In [57]:
sort = {"$sort":{"Id": -1}}
limit = {"$limit": 1}
for idele in transactions.aggregate([sort,limit]):
    print(idele)

{'_id': ObjectId('60100aaf0ae46331748c4ece'), 'Id': 103, 'Name': 'Dylan', 'TransactionId': 'tran4', 'Panier': [{'ItemId': 'a100', 'price': 200}, {'ItemId': 'a110', 'price': 210}], 'Subscriber': True, 'Payment': None, 'Note': 'Payment is Null'}


### Q-5. Find the max price (Transaction.price) ?

In [60]:
unwi = {"$unwind" : "$Panier"}
proj = {"$project" : {"Panier.price":1,"_id":0}}
sort = {"$sort":{"Panier.price": -1}}
limit = {"$limit": 1}
for maxx in transactions.aggregate([unwi,proj,sort,limit]):
    print(maxx)

{'Panier': {'price': 210}}


### Q : Peut-on faire un distinct sur plusieurs key ?

In [62]:
for nam in restaurants.distinct("name"):
    print(nam)


#1 Garden Chinese
#1 Me. Nick'S
#1 Sabor Latino Restaurant
$1.25 Pizza
''U'' Like Chinese Restaurant
''W'' Cafe
'Wichcraft
(Lewis Drug Store) Locanda Vini E Olii
(Library)  Four & Twenty Blackbirds
(Public Fare) 81St Street And Central Park West (Delacorte Theatre)
/ L'Ecole
002 Mercury Tacos Llc
1 2 3 Burger Shot Beer
1 Banana Queen
1 Buen Sabor
1 Darbar
1 East 66Th Street Kitchen
1 Oak
1 Or 8
1 Stop Patty Shop
1.5 Galbi Corp
10 Devoe
10 Points Ktv
100 Fun
100% Patacon Cachapa Yaroa
100% Smoothies & Empanadas
1001 Nights
1001 Nights Cafe
1005 Catering
101 Cafe
101 Deli
101 Restaurant And Bar
102 Noodles Town Restaurant
1020 Bar
1028 Bar & Restaurant El Salvadoreno 
104-01 Foster Avenue Coffee Shop(Ups)
1061 Catering Llc
107 West Restaurant
108 Fast Food Corp
108 Lounge - Club 108
1081 Fulton
10Th Avenue Cookshop
10Th Avenue Pizza & Cafe
11 Meat Shop
11 Street Cafe
111 Restaurant
1174 Fulton Cuisine, Halal Food
12 Chairs
12 Chairs Cafe
12 Corazones Restaurant & Bar
12 Corners
12 Corne

Bamboo Garden Chinese Restaurant
Bamboo Garden Restaurant
Bamboo House
Bamboo Lounge & Grill
Bamboo Rest
Bamboo Restaurant
Bamboo Sushi
Bamboo Tropical
Bamboo Ya
Bamiyan Persian Cusine
Bamonte'S Restaurant
Banana Boat Restaurant
Banana King
Banana Leaf Restaurant
Banc Cafe
Bangal Curry
Bangia Bar & Lounge
Bangkok 2 Thai
Bangkok Grand Palace
Bangkok Heights
Bangkok House
Bangkok Tasty
Bangkok Thai Cuisine
Bangkok Thai House
Bangla Garden Restaurant
Banh Mi Saigon Bakery
Banh Mi Veitnamese Sandwiches
Banh Mi Zon
Banhmigos
Bani Restaurant Llc
Bank Of America Cafe
Bank Of America Executive Dinning Room
Bank Of New York
Bank Of New York Cafe
Bank Of Nova Scotia Cafe
Bank Of Tokyo
Bankok Cuisine
Bann
Banquet Kitchen (Marriott)
Banshee Pub
Bantam Bagels
Banter
Bantry Bay Publick House
Bany Ten 57 Asian Fusion
Banzobar
Bao Ding
Bao Lai'S
Baoguette Pho Sure
Baohaus
Bar
Bar    1849     Grill
Bar  Sardine
Bar & Grill 43
Bar & Restaurant El Salvadoreno
Bar 13
Bar 131
Bar 21 Inc
Bar 245
Bar 360
Bar

Brewski'S Bar & Grill
Brgr
Brian Dempsey'S Ale House
Briarwood Cafe
Bricco Restaurant
Briciola
Brick Oven Bread
Brick Oven Bread Berikoni
Brick Pizzeria/Nathan'S Famous Hot Dogs
Brickhouse Pizzeria
Bricklane Curry House
Bricktown Bagels & Cafe
Brickyard Gastro Pub
Bricolage
Bridge Coffee Shop
Bridge Pub
Bridges Bar
Bridgeview Diner
Bridies Bar And Grill
Brigadeiro Bakery
Bright Star Chinese Food
Brik Bar Lounge & Kitchen
Brindle Room
Brinkley'S
Brinkley'S Station
Brio Restaurant
Brioso Ristorante
Brisas Del Caribe
Brisas Del Mar Restaurant
Brisas Del Mar Seafood Market
Brisas Express Restaurant
Bristle And Creme
Britain Indian Restaurant
Brittanya 54Th
Brk
Broadhurst Theater
Broadwalk Billiards
Broadway Au Lait
Broadway Bagel
Broadway Bakery
Broadway Bakery & Pizzeria
Broadway Cafe
Broadway China Station
Broadway Chinese Kitchen
Broadway Chinese Seafood
Broadway Comedy Club
Broadway Deli & Bagel'S
Broadway Dive
Broadway Gourmet
Broadway Gourmet Deli & Grill
Broadway Grill
Broadway Joe'

Choza Taqueria
Chris Caribbean & American Restaurant
Chris Restaurant
Chris Super Deli
Christ Coffee Shop & Deli
Christie'S
Christie'S Pub
Christina'S Restaurant
Christine'S Restaurant
Christmas Karaoke
Christopher'S Palace
Christos Gyro & Souvlaki
Christos Steakhouse
Chu'S Gourmet
Chuanshi # 17
Chubby Burger
Chubby Burgers & Chicken
Chubby Burgers Chicken & Pizza
Chubby'S Chicken And Burgers
Chuck E Cheese'S
Chuck E. Cheese'S
Chui Hong Yuan Restaurant
Chui'S Garden
Chuko
Chun Buro Restaurant
Chung -Ah Restaurant
Chung Ching
Chung Chun Kitchen
Chung Hing Chinese Restaurant
Chung Ki Wa Restaurant
Chung Moo Roll Rice & Dongas
Chung Wah
Church Cafe Wine Bar
Church Publick
Church Street Kitchen
Church Street Tavern
Churn Llc
Churrascaria Plataforma
Chutney Kitchen
Chuvoy'S Restaurant & Bar
Ciao Bella
Ciao Bella Gelateria
Ciao Bella Napoli
Ciao For Now
Cibao
Cibao Restaurant
Cibar - Lady Mendl'S Tea Salon
Cibo
Cibo E Vino
Cibo Express
Cibo Express (Baggage Pre-Security)
Cibo Express Gourmet

Dell'Anima
Dellarocco'S Of Brooklyn
Delly'S Place Restaurant & Fritaille
Delmar Pizzeria
Delmonico Gourmet
Delmonico'S Kitchen
Delmonicos
Delmy Food
Delphins Foods
Delroy'S
Delroy'S Cafe
Delta Grill
Delta Kitchen (In Seat Svc Bar 1)
Delta Sky Club
Delta Sky Club (Bartender Service Terminal D Delta Departure)
Deluge Restaurant
Deluxe
Demarchelier Restaurant
Dempsey'S Pub
Den
Denino'S Pizzeria Tavern
Denivan Coffee Shop
Deniz Restaurant
Denize'S Creole Corner
Dennis' Place
Denny'S
Denny'S Pub
Deno'S Snack Bar/Famiglia
Deno'S Sweet Shoppe
Der Kommissar
Der Krung
Dera Restaurant
Desantos Restaurant
Desert Rose Cafe & Lounge
Desi Deli Punjabi Dhaba
Desi Galli
Desi Grill Indian Cuisine
Desi Shack
Desmond'S Tavern
Desmonds Steakhouse And Grill
Desnuda
Desnuda Cevicheria
Despierta Con Energia/Herbalife
Desseo Night Club
Dessert Club, Chikalicious
Dessert Palace
Destefano'S Steakhouse
Destination
Destiny Cafe
Destiny/North River Lobster
Desy'S Clam Bar Restaurant
Detoxx Bar
Deum
Deutsche Bank
D

...

Rubens
Rubens Pizzeria-El Pollo Peruvian Cuisine
Rubi Rosa Restaurant & Bar
Rubirosa Pizza & Ristorante
Ruby Falls Nightlife Cafe
Ruby Foo'S Times Square
Ruby Kitchen Chan Inc
Ruby Tuesday
Ruby'S
Ruchi Indian Cuisine
Rucola
Ruddy And Dean
Rudy'S Bakery
Rudy'S Bar & Grill
Rudy'S Deli Restaurant
Rue 57
Ruffles Bar
Rugova Bar & Grill
Ruinas De Copan
Rumba Supper Club
Rumours
Runner & Stone
Running Cool Ii Restaurant
Running Cool Restaurant
Runway69
Rural Chinese Restaurant
Rush Tapas Bar Inc
Russ & Daughters Cafe
Russ Pizza
Russia Vodka Room
Russian Baths
Russian Samovar
Russian Turkish Baths
Russo'S Bakery
Russo'S On The Bay
Rustico
Rustik 471
Rusty Mackerel
Rusty'S Bar
Rusty'S Flavor
Ruth M Liedys Shore Inn
Ruth'S Chris Steak House
Ruthie'S
Ruthys Bakery & Cafe
Rw Prime
Rw Prime Service Bar
Ryan Maguire'S Bar & Restaurant
Ryan'S Daughter Cafe
Rye
Ryehouse
Rythym & Booze
S & A West Indian Restaurant
S & J Kitchen
S & J Restaurant And Bakery
S & P Internet Cafe
S & S Caribbean Cuisine And

Spiros Restautrant
Spitzer'S Corner
Splendid China
Splitty
Spolem
Spolini'S
Spoon Bread Catering
Spoons
Sporting Club Gjoa
Sports Center At Chelsea Piers (Sushi Bar)
Sports Center Cafe
Sports Luncheonette
Sports Mania Bar & Grill
Spot
Spoto'S Restaurant
Spreadhouse Coffee & Art, Llc
Spreads
Spring
Spring Fish Village
Spring Garden
Spring Garden Chinese Restaurant
Spring Garden Restaurant
Spring Gardens Restaurant
Spring Hill Hotel
Spring Natural Kitchen
Spring St. Natural Restaurant
Springfield & La Bari Pizzeria
Springfield Diner
Springhill Suite
Sprinkles Cupcakes
Sprinkles Ice Cream
Spritzenhaus
Spunto
Spuyten Duyvul
Spyglass
Squadz
Square Diner
Square Pizza
Squire Diner
Sqz Bar
Srb Brooklyn
Sri Thai Cafe
Sripraphai Thai Restaurant
Sriracha
Ssam
St Bess Restaurant
St George Tower
St Giles Hotel - Icon/The Court
St James Gate
St James Theatre
St John'S Express
St John'S University Faculty Dining Room
St John'S University Law School
St John'S University Library Cafe
St John'S Universi

The Diner
The Ding Dong Lounge
The Dish Restaurant
The District
The Door
The Dove Parlour
The Drink
The Dub Pie Shop
The Dubliner
The Duck
The Dugout
The Dumplin Shop
The Duplex
The Dutch
The Eagle
The Eagle Cafe
The East Pole
The Eddy
The Edge
The Edge Bar
The Elk
The Ellington
The Elm
The Emerson
The End Zone Bar
The Esplanade
The Everett Cafe/Starbucks-Teachers College
The Faculty Club (Columbia University)
The Faculty House
The Falafel Shop
The Falls Cafe
The Family Restaurant
The Famous Billy'S Pizza & Pasta
The Famous Chicken Place
The Famous Emilio'S Pizza
The Famous Jimbo'S Hamburger Palace
The Farm On Adderley
The Fat Radish
The Fifth Estate
The Financier Patisserie
The Finch
The Fitz
The Five Spot
The Flat
The Flatiron Room
The Flying Lobster
The Flying Puck
The Folly
The Food Hut
The Fork Bar & Grill
The Four Faced Liar
The Four Seasons
The Fourth American Brasserie
The Friedman Theatre
The Frog'S Crown Bakery
The Full Shilling
The Gaf
The Gander Bar Room And Restaurant
The 

Tribeca Pizzeria
Tribeca Rooftop
Tribeca Taphouse
Tribeca Tavern
Tribeca Treats
Tribeca'S Cornerstone
Triestes Restaurant
Trinciti Kitchen
Trinciti Roti Shop & Restaurant
Trini Breakfast Shed Ii
Trini Delight
Trini Delite Roti Shop
Trini Gul Original Roti  & West Indian Food
Trini Gyul Original Roti & West Indian Food
Trini Home Restaurant
Trini Roti House And Bakery
Trinidad Ali'S Roti Shop #2
Trinidad Golden Place
Trinity Place Bar & Restaurant
Trinity Pub
Trio Pizza Grill
Triomphe
Triona'S
Triona'S Bar Restaurant
Triple 8 Restaurant
Triple A Diner
Triple C Restaurant And Cuisine
Triple Crown Cafe
Triple Crown Diner
Triple D'S Place
Triple Shot World Atlas
Triple T Pizza & Burgers
Tripoli Restaurant
Tripp & Cooper Cafe
Trix
Troost
Trophy
Tropic Juice Bar & Grill
Tropic Pollo
Tropical 128
Tropical Bar
Tropical Bar Restaurant
Tropical Coffee Shop
Tropical Fantasy Bar And Lounge
Tropical Flavors
Tropical Grill
Tropical Grill & Restaurant
Tropical Grill Restaurant
Tropical House Baking C

Yogo Monster
Yogorino
Yogurt & Company
Yogurt City
Yogurt Couture
Yogurt Down Under
Yogurt In Love
Yogurt La Crepe
Yogurt Y23 Inc
Yogurtberry
Yogurtland
Yokohama Japanese Restaurant
Yola'S Cafe
Yolanda Pizzeria Restaurant
Yolanda Restaurant
Yoli Restaurant
Yolie'S Bar & Restaurant
Yomaris Restaurant
Yomi Station
Yonah Shimmels Knishes
Yonekichi
Yong Sheng Chinese Restaurant
Yong Sheng Restaurant
Yoo'S Evergreen Salad Bar And Sandwiches
Yooberry Frozen Yogurt
Yooglers Frozen Yogurt
Yopparai
Yorganic
Yorganic/Caffe Bene
York Chan Restaurant
York College Cafeteria
Yorkafe
Yorkville Creperie/Saloon
Yoshi
Yoshi Modern Japanese Cuisine
Yosmira Restaurant
You Yi Chinese Restaurant
Youge Yogurt
Your Bakery
Your Daley Bread
Your House
Your House Cafe
Yours & Mine Restaurant
Yoyo Dessert# 6
Yoyo Fritaille
Yoz Shanghai
Yoz Shanghai (Booth 20)
Yseb
Yu Garden Dumpling House
Yu King Bakery
Yu Sushi
Yu Sushi Bar
Yuba
Yuca Bar & Restaurant
Yue Lai Bakery
Yuji Ramen
Yuka Japanese Restaurant
Yum Yum Bak